In [3]:
!pip install numpy pandas scikit-learn matplotlib nltk

In [4]:
!python -m pip install --upgrade pip

In [5]:
!pip install kagglehub

In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\pgcho\.cache\kagglehub\datasets\lakshmi25npathi\imdb-dataset-of-50k-movie-reviews\versions\1


In [7]:
import os

dataset_path = r"C:\Users\pgcho\.cache\kagglehub\datasets\lakshmi25npathi\imdb-dataset-of-50k-movie-reviews\versions\1"
print(os.listdir(dataset_path))

['IMDB Dataset.csv']


In [8]:
import pandas as pd

file_path = os.path.join(dataset_path, "IMDB Dataset.csv")  # Adjust filename if different
df = pd.read_csv(file_path)

# Show first 5 rows
print(df.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [9]:
print(df.isnull().sum())

review       0
sentiment    0
dtype: int64


In [10]:
df.dropna(inplace=True)

In [11]:
df['review'] = df['review'].str.lower()

In [12]:
import string

df['review'] = df['review'].str.translate(str.maketrans("", "", string.punctuation))

In [13]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

df['review'] = df['review'].apply(lambda x: " ".join(word for word in x.split() if word not in stop_words))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pgcho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
df['review'] = df['review'].apply(lambda x: x.split())

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)  # Use top 5000 words
X = vectorizer.fit_transform(df['review'].apply(lambda x: " ".join(x)))  # Convert list back to string

print(X.shape)  # Should print (50000, 5000) -> 50K reviews, 5K features

(50000, 5000)


In [16]:
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment'], test_size=0.2, random_state=42)

print(f"Training Samples: {X_train.shape[0]}, Testing Samples: {X_test.shape[0]}")

Training Samples: 40000, Testing Samples: 10000


In [18]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)  # Train the model

LogisticRegression()

In [19]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(X_test)  # Predict on test data
accuracy = accuracy_score(y_test, y_pred)

print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.89


In [20]:
def predict_sentiment(review):
    review = review.lower().translate(str.maketrans("", "", string.punctuation))  # Clean text
    review = " ".join(word for word in review.split() if word not in stop_words)  # Remove stop words
    review_vectorized = vectorizer.transform([review])  # Convert to numbers
    prediction = model.predict(review_vectorized)[0]  # Get prediction
    return "Positive 😊" if prediction == 1 else "Negative 😞"

# Try a sample review
print(predict_sentiment("I absolutely loved this movie! The acting was great and the story was fantastic!"))
print(predict_sentiment("This was a terrible movie. The plot was boring and the acting was bad."))

Positive 😊
Negative 😞


In [21]:
!pip install tensorflow keras

In [22]:
!pip uninstall -y tensorflow h5py numpy

Found existing installation: tensorflow 2.16.1
Uninstalling tensorflow-2.16.1:
  Successfully uninstalled tensorflow-2.16.1
Found existing installation: h5py 3.12.1
Uninstalling h5py-3.12.1:
  Successfully uninstalled h5py-3.12.1
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4


You can safely remove it manually.
You can safely remove it manually.


In [23]:
!pip install numpy==1.26.4
!pip install h5py==3.12.1
!pip install tensorflow==2.16.1

  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras 3.8.0 requires h5py, which is not installed.
tensorflow-intel 2.16.1 requires h5py>=3.10.0, which is not installed.
bezier 2024.6.20 requires numpy>=2.0.0, but you have numpy 1.26.4 which is incompatible.
blis 1.0.1 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


  Using cached h5py-3.12.1-cp311-cp311-win_amd64.whl.metadata (2.5 kB)
Using cached h5py-3.12.1-cp311-cp311-win_amd64.whl (3.0 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.16.1 which is incompatible.


  Using cached tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata (3.5 kB)
Using cached tensorflow-2.16.1-cp311-cp311-win_amd64.whl (2.1 kB)


In [30]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [31]:
max_length = 100
X_train_pad = pad_sequences(X_train.toarray(), maxlen=max_length, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test.toarray(), maxlen=max_length, padding='post', truncating='post')

print(f"Shape of padded training data: {X_train_pad.shape}")
print(f"Shape of padded test data: {X_test_pad.shape}")

Shape of padded training data: (40000, 100)
Shape of padded test data: (10000, 100)


In [32]:
# Cell 2: Enhanced Model Architecture
model = Sequential([
    Input(shape=(100,)),
    Embedding(input_dim=5000, output_dim=128),
    Dropout(0.3),
    Bidirectional(LSTM(256, return_sequences=True)),
    Dropout(0.3),
    Bidirectional(LSTM(128)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# Compile with optimized settings
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', 
              optimizer=optimizer, 
              metrics=['accuracy'])

In [33]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape   ┃ Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ embedding_1         │ (None, 100,    │ 640,000 │
│ (Embedding)         │ 128)           │         │
├─────────────────────┼────────────────┼─────────┤
│ dropout (Dropout)   │ (None, 100,    │       0 │
│                     │ 128)           │         │
├─────────────────────┼────────────────┼─────────┤
│ bidirectional       │ (None, 100,    │ 788,480 │
│ (Bidirectional)     │ 512)           │         │
├─────────────────────┼────────────────┼─────────┤
│ dropout_1 (Dropout) │ (None, 100,    │       0 │
│                     │ 512)           │         │
├─────────────────────┼────────────────┼─────────┤
│ bidirectional_1     │ (None, 256)    │ 656,384 │
│ (Bidirectional)     │                │         │
├─────────────────────┼────────────────┼─────────┤
│ dropout_2 (Dropout) │ (None, 256)    │       0 │
├─────────────────────┼────────────────┼─────────┤
│ dense_1 (Dense)     │ (None, 64)     │  16,448 │
├─────────────────────┼────────────────┼─────────┤
│ dropout_3 (Dropout) │ (None, 64)     │       0 │
├─────────────────────┼────────────────┼─────────┤
│ dense_2 (Dense)     │ (None, 1)      │      65 │
└─────────────────────┴────────────────┴─────────┘

 Total params: 2,101,377 (8.02 MB)

 Trainable params: 2,101,377 (8.02 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Cell 3: Training with Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)
]

history = model.fit(
    X_train_pad, y_train,
    epochs=20,
    batch_size=64,
    validation_split=0.2,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 429s 846ms/step - accuracy: 0.4958 - loss: 0.6942 - val_accuracy: 0.5000 - val_loss: 0.6934 - learning_rate: 0.0010
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 403s 805ms/step - accuracy: 0.4974 - loss: 0.6934 - val_accuracy: 0.5000 - val_loss: 0.6932 - learning_rate: 0.0010
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 513s 1s/step - accuracy: 0.4954 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932 - learning_rate: 0.0010
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 467s 835ms/step - accuracy: 0.4988 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6934 - learning_rate: 0.0010
Epoch 5/20
107/500 ━━━━━━━━━━━━━━━━━━━━ 9:18 1s/step - accuracy: 0.5012 - loss: 0.6932

In [ ]:
# Cell 4: Evaluation and Visualization
test_loss, test_accuracy = model.evaluate(X_test_pad, y_test, verbose=0)
print(f"Test accuracy: {test_accuracy:.4f}")

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Cell 5: Prediction Function
def predict_sentiment_lstm(review):
    review_cleaned = preprocess_text(review)
    review_vectorized = vectorizer.transform([review_cleaned])
    review_padded = pad_sequences(review_vectorized.toarray(), maxlen=max_length, padding='post', truncating='post')
    prediction = model.predict(review_padded)[0][0]
    return "Positive 😊" if prediction > 0.5 else "Negative 😞"

# Test predictions
test_reviews = [
    "This movie was absolutely fantastic! Great acting and amazing plot!",
    "Terrible waste of time. Poor acting and boring story.",
    "An interesting movie with some good moments but also some flaws."
]

for review in test_reviews:
    print(f"Review: {review}")
    print(f"Sentiment: {predict_sentiment_lstm(review)}\n")